\# **Notebook for reproducing CCAM results**

In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

2.2.1+cu121
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install scipy==1.8.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.4.23 requires scipy>=1.9, but you have scipy 1.8.1 which is incompatible.
jaxlib 0.4.23+cuda12.cudnn89 requires scipy>=1.9, but you have scipy 1.8.1 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.24.4 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
from torch_geometric.datasets import Planetoid, Amazon,WebKB,FB15k_237,HeterophilousGraphDataset,WikipediaNetwork,CitationFull,Actor
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.utils import homophily,add_self_loops, is_undirected,to_networkx,from_networkx,to_undirected, to_dense_adj, dense_to_sparse
import time


An example with the Texas dataset

In [80]:

#dataset = Planetoid(root='data/Planetoid', name='cora', transform=NormalizeFeatures())
#dataset = Amazon(root='data/amazon', name='Computers', transform=NormalizeFeatures())
#dataset = WebKB(root='data/WebKB', name='texas',transform = NormalizeFeatures())
#dataset = CitationFull(root='data/Citeseer', name='Citeseer', to_undirected = False)#
dataset = WikipediaNetwork(root='data/WikipediaNetwork', name='squirrel')
#dataset = HeterophilousGraphDataset(root='data/Roman-empire', name='Roman-empire')
#dataset = Actor(root='data/Actor')

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.
data.y
print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')


print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

data_und = to_undirected(data.edge_index)
data_und,_= add_self_loops(data_und, num_nodes=data.num_nodes)


Processing...



Dataset: WikipediaNetwork():
Number of graphs: 1
Number of features: 2089
Number of classes: 5

Data(x=[5201, 2089], edge_index=[2, 217073], y=[5201], train_mask=[5201, 10], val_mask=[5201, 10], test_mask=[5201, 10])
Number of nodes: 5201
Number of edges: 217073
Average node degree: 41.74
Number of training nodes: 24960
Training node label rate: 4.80
Has isolated nodes: False
Has self-loops: True
Is undirected: False


Done!


In [ ]:
#rom bf_curvature import *
'''
!cp /content/drive/MyDrive/curvaadj/FAUX_bf_curvature.py bf_curvature.py
import bf_curvature

import numpy as np
start_time = time.time()
C = bf_curvature.balanced_forman_curvature(data_und,data.num_nodes,False)
C.shape
C.max()
print(time.time() - start_time)
mat_ricci=np.zeros((data.num_nodes,data.num_nodes))
mat_riccim=np.zeros((data.num_nodes,data.num_nodes))

C.max()

torch.where(adj[1] > 0)[0]
np.sum(mat_riccim)
#mat_ricci = np.where(C >= mean_C, C, 0)
#mat_riccim = np.where(C <= mean_C, C, 0)


Utilisation de la courbure de Forman Balanced (TOPPING ICLR 2022) (Besoin de telecharger les matrices de courbure pré-calculer )


---



---






In [ ]:

import pickle
C = open('/content/drive/MyDrive/curvaadj/courbure-bfc-cora.pk','rb') #Récuperation de la matrice de courbure pré calculer en amont
C =pickle.load(C)

In [ ]:
values = C[data.edge_index[0], data.edge_index[1]]
values.shape
moyenne_arretes = torch.mean(values)
torch.nanmean(values)
print(moyenne_arretes)
mat_ricci = np.where(C.cpu() >= moyenne_arretes.cpu(), C.cpu(), 0)
mat_riccim = np.where(C.cpu() <= moyenne_arretes.cpu(), C.cpu(), 0)

tensor(-0.3010, device='cuda:0')


In [ ]:
new_adj,_ = dense_to_sparse(new_adj)
new_adj,_= add_self_loops(new_adj, num_nodes=data.num_nodes)




---



Utilisation de la courbure de Augmented Forman et Olliver


In [4]:
!pip install GraphRicciCurvature
from GraphRicciCurvature.OllivierRicci import OllivierRicci
import networkx as nx
# load GraphRicciCuravture package
from GraphRicciCurvature.OllivierRicci import OllivierRicci
from GraphRicciCurvature.FormanRicci import FormanRicci


import networkx as nx
import numpy as np
import math
%matplotlib inline
import matplotlib.pyplot as plt

# to print logs in jupyter notebook
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

# load GraphRicciCuravture package
from GraphRicciCurvature.OllivierRicci import OllivierRicci
from GraphRicciCurvature.FormanRicci import FormanRicci

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 31.1 MB/s eta 0:00:00


Calculate curvature according to Ollivier

In [54]:
def mean_courbure(G,curvature="formanCurvature") : #formanCurvature/ricciCurvature
    total_forman_curvature = 0
    nombre_d_elements = 0
    for sommet, voisins in G_curv.adj.items():
        for voisin, attributs in voisins.items():
            forman_curvature = attributs.get(curvature, None)
            if forman_curvature is not None:
                total_forman_curvature += forman_curvature
                nombre_d_elements += 1
    if nombre_d_elements > 0:
        moyenne_forman_curvature = total_forman_curvature / nombre_d_elements
        print("Moyenne des valeurs de formanCurvature :", moyenne_forman_curvature)
    else:
        print("Aucune valeur de formanCurvature trouvée.")
    return moyenne_forman_curvature

In [85]:
#curvature = "ricciCurvature"/formanCurvature

def show_results(G):
    # Print the first five results
    print("Karate Club Graph, first 5 edges: ")
    for n1,n2 in list(G.edges())[:5]:
        print("Ollivier-Ricci curvature of edge (%s,%s) is %f" % (n1 ,n2, G[n1][n2]["formanCurvature"]))

    # Plot the histogram of Ricci curvatures
    plt.subplot(2, 1, 1)
    ricci_curvtures = nx.get_edge_attributes(G, "formanCurvature").values()

    values = list(ricci_curvtures)
    values.sort()
    print(values)

    # Calcul du premier quartile (Q1)
    n = len(values)

    squash_index = int(n * 0.1)
    print(squash_index)  # Index correspondant au premier deciles
    q1 = values[squash_index]


    n = len(values)

    smooth_index = int(n * 0.9)
    print(smooth_index)  # Index correspondant au premier deciles

     # Index correspondant au derniere deciles
    q10 = values[smooth_index]


    return q1,q10


Q1,Q10 = show_results(G_curv)

12.0

In [86]:
A = torch.squeeze(to_dense_adj(data_und)).numpy()
G=nx.Graph(A)
start_time = time.time()
#curvature = "ricciCurvature" #Coubure d'ollivier
#orc = OllivierRicci(G, alpha=0.5, verbose="TRACE")#Coubure d'ollivier
curvature = "formanCurvature" #Coubure de forman
orc = FormanRicci(G)
start_time = time.time()
orc.compute_ricci_curvature()
G_curv = orc.G.copy()
print(time.time() - start_time)


mat_ricci=np.zeros((data.num_nodes,data.num_nodes))
mat_riccim=np.zeros((data.num_nodes,data.num_nodes))
mat_ricci_ent=np.zeros((data.num_nodes,data.num_nodes))

mean_mr = mean_courbure(G)


for a,i in enumerate(np.unique(G_curv)):
    for j in list(G_curv.neighbors(a)):
        if G_curv[i][j][curvature]>=Q1 :
            mat_ricci[a][j]=G_curv[i][j][curvature]
        if G_curv[i][j][curvature]<=Q1 :
            mat_riccim[a][j]=G_curv[i][j][curvature]

209.5550479888916
Moyenne des valeurs de formanCurvature : -392.22435303795675


Filtrer la matrice d'adjancence selon la courbure

In [87]:
curvp = torch.Tensor(mat_ricci)
curvm = torch.Tensor(mat_riccim)
curvp = curvp.nonzero().t().contiguous()
curvm = curvm.nonzero().t().contiguous()

edge_index_curvp,_ = add_self_loops(curvp, num_nodes=data.num_nodes)
edge_index_curvm,_ = add_self_loops(curvm, num_nodes=data.num_nodes)

edge_index_curvp = to_undirected(edge_index_curvp)
edge_index_curvm = to_undirected(edge_index_curvm)
is_undirected(edge_index_curvm)

True

Measurement of homophily on edges presented by Zhu et al. and the measurement Curvature-Constrained homophily

In [88]:
print(homophily(data_und, data.y))
print(homophily(edge_index_curvp, data.y))
print(homophily(edge_index_curvm, data.y))


0.23246784508228302
0.2294272780418396
0.34220656752586365


Possibly building a two-hop based on curvature.

In [ ]:
from torch_geometric.utils import add_self_loops
num_nodes = data.num_nodes
new_edge_index = []

def build_two_hop(mat_adj):

    for node in range(num_nodes):
        neighbors = mat_adj[1][mat_adj[0] == node]

        for neighbor in neighbors:
            neighbors_of_neighbor = mat_adj[1][mat_adj[0] == neighbor]

            neighbors_of_neighbor = neighbors_of_neighbor[neighbors_of_neighbor != node]

            for neighbor_of_neighbor in neighbors_of_neighbor:
                new_edge_index.append([node, neighbor_of_neighbor.item()])

    two_hop_tensor = torch.tensor(new_edge_index, dtype=torch.long).t()
    two_hop_tensor,_ = add_self_loops(two_hop_tensor, num_nodes=data.num_nodes)
    return two_hop_tensor

two_hop_tensor = build_two_hop(edge_index_curvm)#edge_index_curvm/curvp => matrice d'adjance négative/positive
print(homophily(two_hop_tensor, data.y))
is_undirected(two_hop_tensor)
two_hop_tensor.shape

0.5602510571479797


torch.Size([2, 11311])

GCN

In [90]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x.to(device), edge_index_curvp.to(device))
        x = x.relu()
        x = F.dropout(x.to(device), p=0.5, training=self.training)
        x = self.conv2(x.to(device), edge_index_curvp.to(device))
        return x.to(device)

GAT

In [46]:
from torch_geometric.nn import GATConv
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class GAT(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = GATConv(dataset.num_features, hidden_channels,heads=8,dropout= 0.5)
        self.conv2 = GATConv(hidden_channels*8, dataset.num_classes,heads=1,dropout= 0.5)

    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x.to(device), two_hop_tensor.to(device)))
        x = F.dropout(x.to(device), p=0.5, training=self.training)
        x = self.conv2(x.to(device), two_hop_tensor.to(device))
        return x.to(device)


In [48]:
criterion = torch.nn.CrossEntropyLoss()

def train(t,v):
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x.to(device), data.edge_index.to(device))  # Perform a single forward pass.
      loss = criterion(out[data_train_mask], data.y.to(device)[data_train_mask])  # Compute the loss solely based on the training nodes.
      val_loss = criterion(out[v].to(device), data.y.to(device)[v])  # Compute the loss solely based on the val nodes.
      loss.backward() # Derive gradients.
      #val_loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss,val_loss
def test(mask):
      model.eval()
      out = model(data.x.to(device), data.edge_index.to(device))
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      correct = pred[mask] == data.y.to(device)[mask]  # Check against ground-truth labels.
      acc = int(correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
      return acc

In [91]:
import time
import numpy as np
ramdom = True
moy = []
ep = []
T = []
p_m =[]
nb_runs = 10
hidden_channels = 48
lr = 0.005
weight_decay = 5e-5
moy = []
ep = []
for runs in range(nb_runs):
    model = GCN(hidden_channels=hidden_channels).to(device)
    #model = GAT(hidden_channels=hidden_channels).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    criterion = torch.nn.CrossEntropyLoss()
    print("runs", runs)
    start_time = time.time()
    shuffled_indices = torch.randperm(len(data.y))
    if ramdom == True :


        indices = [False for i in range(len(data.y))]
        mask_train = [True if i in shuffled_indices[:20*(torch.max(data.y)+1)] else False for i in range(len(indices))]
        mask_val = [True if i in shuffled_indices[20*(torch.max(data.y)+1):20*(torch.max(data.y)+1)+500] else False for i in range(len(indices))]
        mask_test = [True if i in shuffled_indices[20*(torch.max(data.y)+1)+500:20*(torch.max(data.y)+1)+1500] else False for i in range(len(indices))]

        data_train_mask = torch.tensor(mask_train)
        data_val_mask = torch.tensor(mask_val)
        data_test_mask = torch.tensor(mask_test)


    indices = [False for i in range(len(data.y))]
    mask_train = [True if i in shuffled_indices[:int(0.6*len(data.y))] else False for i in range(len(indices))]
    mask_val = [True if i in shuffled_indices[int(0.6*len(data.y)):int(0.8*len(data.y))] else False for i in range(len(indices))]
    mask_test = [True if i in shuffled_indices[int(0.8*len(data.y)):len(data.y)] else False for i in range(len(indices))]


    data_train_mask = torch.tensor(mask_train)
    data_val_mask = torch.tensor(mask_val)
    data_test_mask = torch.tensor(mask_test)

    best_val_acc = 0
    i=0
    for epoch in range(1, 2001):
        loss,val_loss = train(data_train_mask,data_val_mask)
        val_acc = test(data_val_mask)
        train_acc = test(data_train_mask)
        test_acc = test(data_test_mask)
        if val_acc > best_val_acc :
          best_val_acc = val_acc
          Test_acc = test_acc
          i=0
        i=i+1
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},Train_acc: {train_acc:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f},test_acc: {test_acc:.4f}')
        #print(i)
        if i == 100 :
          break

    T.append(time.time() - start_time)
    print(f'Test Accuracy: {Test_acc:.4f}')
    moy.append(Test_acc)
    ep.append(epoch)
print("-------------------------" )
print("mod" , model)
print("hidden_channels" , hidden_channels)
print("weight_decay" , weight_decay)
print("lr" , lr)
print("moyenne" , np.mean(moy))
print("std" , np.std(moy))
print("temps" , np.mean(T))
print((2*(np.std(moy))/(10))*100)


print("-------------------------" )



runs 0
Epoch: 001, Loss: 1.6108,Train_acc: 0.2071, val_loss: 1.6108, val_acc: 0.1865,test_acc: 0.1921
Epoch: 002, Loss: 1.5860,Train_acc: 0.2074, val_loss: 1.5872, val_acc: 0.1865,test_acc: 0.1921
Epoch: 003, Loss: 1.5791,Train_acc: 0.2074, val_loss: 1.5850, val_acc: 0.1865,test_acc: 0.1921
Epoch: 004, Loss: 1.5703,Train_acc: 0.2170, val_loss: 1.5797, val_acc: 0.1942,test_acc: 0.1950
Epoch: 005, Loss: 1.5610,Train_acc: 0.2571, val_loss: 1.5721, val_acc: 0.2231,test_acc: 0.2219
Epoch: 006, Loss: 1.5490,Train_acc: 0.2955, val_loss: 1.5579, val_acc: 0.2510,test_acc: 0.2440
Epoch: 007, Loss: 1.5393,Train_acc: 0.3356, val_loss: 1.5522, val_acc: 0.2817,test_acc: 0.2824
Epoch: 008, Loss: 1.5300,Train_acc: 0.3554, val_loss: 1.5465, val_acc: 0.3067,test_acc: 0.3141
Epoch: 009, Loss: 1.5217,Train_acc: 0.3631, val_loss: 1.5382, val_acc: 0.3106,test_acc: 0.3170
Epoch: 010, Loss: 1.5092,Train_acc: 0.3769, val_loss: 1.5321, val_acc: 0.3183,test_acc: 0.3266
Epoch: 011, Loss: 1.4985,Train_acc: 0.3891,

KeyboardInterrupt: 

In [ ]:
import time
import numpy as np
ramdom = True
moy = []
ep = []
T = []
p_m =[]
nb_runs = 50
hidden_channels = 48
lr = 0.005
weight_decay = 5e-5
moy = []
ep = []
for runs in range(nb_runs):
    #model = GCN(hidden_channels=hidden_channels).to(device)
    model = GAT(hidden_channels=hidden_channels).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    criterion = torch.nn.CrossEntropyLoss()
    print("runs", runs)
    start_time = time.time()
    shuffled_indices = torch.randperm(len(data.y))
    if ramdom == True :


        indices = [False for i in range(len(data.y))]
        mask_train = [True if i in shuffled_indices[:20*(torch.max(data.y)+1)] else False for i in range(len(indices))]
        mask_val = [True if i in shuffled_indices[20*(torch.max(data.y)+1):20*(torch.max(data.y)+1)+500] else False for i in range(len(indices))]
        mask_test = [True if i in shuffled_indices[20*(torch.max(data.y)+1)+500:20*(torch.max(data.y)+1)+1500] else False for i in range(len(indices))]

        data_train_mask = torch.tensor(mask_train)
        data_val_mask = torch.tensor(mask_val)
        data_test_mask = torch.tensor(mask_test)


    indices = [False for i in range(len(data.y))]
    mask_train = [True if i in shuffled_indices[:int(0.6*len(data.y))] else False for i in range(len(indices))]
    mask_val = [True if i in shuffled_indices[int(0.6*len(data.y)):int(0.8*len(data.y))] else False for i in range(len(indices))]
    mask_test = [True if i in shuffled_indices[int(0.8*len(data.y)):len(data.y)] else False for i in range(len(indices))]


    data_train_mask = torch.tensor(mask_train)
    data_val_mask = torch.tensor(mask_val)
    data_test_mask = torch.tensor(mask_test)

    best_val_acc = 0
    i=0
    for epoch in range(1, 2001):
        loss,val_loss = train(data_train_mask,data_val_mask)
        val_acc = test(data_val_mask)
        train_acc = test(data_train_mask)
        test_acc = test(data_test_mask)
        if val_acc > best_val_acc :
          best_val_acc = val_acc
          Test_acc = test_acc
          i=0
        i=i+1
        #print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},Train_acc: {train_acc:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f},test_acc: {test_acc:.4f}')
        #print(i)
        if i == 100 :
          break

    T.append(time.time() - start_time)
    print(f'Test Accuracy: {Test_acc:.4f}')
    moy.append(Test_acc)
    ep.append(epoch)
print("-------------------------" )
print("mod" , model)
print("hidden_channels" , hidden_channels)
print("weight_decay" , weight_decay)
print("lr" , lr)
print("moyenne" , np.mean(moy))
print("std" , np.std(moy))
print("temps" , np.mean(T))
print((2*(np.std(moy))/(10))*100)


print("-------------------------" )
#moyenne 0.6517647058823529 simple
#moyenne 0.66 double
#moyenne 0.6490196078431374 (two hop)


runs 0
Test Accuracy: 0.6447
runs 1
Test Accuracy: 0.6754
runs 2
Test Accuracy: 0.6776
runs 3
Test Accuracy: 0.6491
runs 4
Test Accuracy: 0.6732
runs 5
Test Accuracy: 0.6294
runs 6
Test Accuracy: 0.6908
runs 7
Test Accuracy: 0.6623
runs 8
Test Accuracy: 0.6535
runs 9
Test Accuracy: 0.6711
runs 10
Test Accuracy: 0.6447
runs 11
Test Accuracy: 0.6732
runs 12
Test Accuracy: 0.6338
runs 13
Test Accuracy: 0.6404
runs 14
Test Accuracy: 0.6776
runs 15
Test Accuracy: 0.6228
runs 16


KeyboardInterrupt: 

Calcul du Spectral Gap normalisé

In [ ]:
from torch_geometric.utils import to_dense_adj
def spectral_gap(adj):
    Adj = to_dense_adj(adj.squeeze()).squeeze()
    G = nx.Graph(Adj.numpy())
    eigenvalues = nx.normalized_laplacian_spectrum(G)

    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues_sorted = eigenvalues[sorted_indices]

    spectral_gap = eigenvalues_sorted[1]
    sum_of_eigenvalues = np.sum(eigenvalues_sorted[1:])

    normalized_spectral_gap = spectral_gap / sum_of_eigenvalues
    return normalized_spectral_gap


In [ ]:
after1 = spectral_gap(two_hop_tensor)#edge_index_curvp
before = spectral_gap(data_und)
print("improvement of the spectral gap of", ((after1 - before)/before)*100, "%" )

improvement of the spectral gap of -19.70755316810613 %


In [ ]:
after = spectral_gap(edge_index_curvm)#edge_index_curvp
#before = spectral_gap(data_und)
print("improvement of the spectral gap of", ((after - before)/before)*100, "%" )

improvement of the spectral gap of 63.122031313100145 %


actor

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import pairwise_distances

#releated paper:(AAAI2020) Measuring and Relieving the Over-smoothing Problem for Graph Neural Networks from the Topological View.
#https://aaai.org/ojs/index.php/AAAI/article/view/5747

#the numpy version for mad (Be able to compute quickly)
#in_arr:[node_num * hidden_dim], the node feature matrix;
#mask_arr: [node_num * node_num], the mask matrix of the target raltion;
#target_idx = [1,2,3...n], the nodes idx for which we calculate the mad value;
def mad_value(in_arr, mask_arr, distance_metric='cosine', digt_num=4, target_idx =None):
    dist_arr = pairwise_distances(in_arr, in_arr, metric=distance_metric)

    mask_dist = np.multiply(dist_arr,mask_arr)

    divide_arr = (mask_dist != 0).sum(1) + 1e-8

    node_dist = mask_dist.sum(1) / divide_arr

    #if target_idx.any()==None:
    mad = np.mean(node_dist)
    #else:
        #node_dist = np.multiply(node_dist,target_idx)
        #mad = node_dist.sum()/((node_dist!=0).sum()+1e-8)

    mad = round(mad, digt_num)

    return mad

with torch.no_grad():
  features = model(data.x, edge_index_curvm)
adj_V = to_dense_adj(edge_index_curvm).squeeze().numpy()
mad_value(features.cpu().numpy() , adj_V)

0.1339

In [ ]:
with torch.no_grad():
  features = model(data.x, data_und)
adj_V = to_dense_adj(data_und).squeeze().numpy()
mad_value(features.cpu().numpy() , adj_V)

0.3912